In [87]:
# You have to find the top-10 managers whose subordinates 
# processed the most total number of incoming requests lasting at least five minutes.

import pandas as pd
import numpy as np
from datetime import datetime

logs = pd.read_csv('contacts', names = ['contact_id', 'client_id', 'employee_id','started_dttm', 'finished_dttm'
                                        , 'business_line', 'route_type', 'initiator_id'], sep = '\t')


emploeeys = pd.read_csv('employees', names = ['employee_id', 'manager_id', 'login'], sep = '\t')

print("====== Reading finished =====")

logs['started_dttm'] = pd.to_datetime(logs['started_dttm'])
logs['finished_dttm'] = pd.to_datetime(logs['finished_dttm'])
logs['call'] = (logs['finished_dttm'] - logs['started_dttm']).dt.seconds
logs['in_out'] = logs["employee_id"] == logs["initiator_id"]
logs['in_out'] = logs['in_out'].apply(lambda x: 'outcome' if x is True else 'income')
logs_filtered = logs[(logs['call'] >= 300)&(logs['in_out'] == 'income')]

print("====== Preparing the dataFrame finished =====")

e = pd.merge(emploeeys, 
                  emploeeys,
                  left_on='manager_id',
                  right_on='employee_id',
                  how='inner')

employees_modified = e[['employee_id_x', 'manager_id_y', 'login_y']]
employees_modified = employees_modified.rename(columns={"employee_id_x": "employee_id", "manager_id_y": "manager_id", "login_y":"login"})

logs_filtered_managers = logs_filtered.merge(employees_modified, on='employee_id', how='inner')

with open('mapreduce_abramov_2.tsv', 'w') as f:
    for i in logs_filtered_managers.groupby('login')['call'].count().sort_values(ascending=False).reset_index().values.tolist()[:10]:
            f.write("{0}\t{1}".format(i[0], int(i[1])))
            f.write('\n')
            print("{0}\t{1}".format(i[0], int(i[1])))

====== Reading finished =====
====== Preparing the dataFrame finished =====
j.n.smith	51620
b.yearwood	42356
l.kistner	38205
u.orellano	37268
r.applegarth	35577
l.busk	34639
z.lee	33448
a.anderson	33441
r.lowe	33214
s.i.lee	32678
